# Coherent info

In [1]:
import numpy as np
from collections import Counter
from typing import Tuple, Iterable, Dict, List
from coherentinfo.moebius import MoebiusCode
from coherentinfo.linalg import (finite_field_gauss_jordan_elimination,
                                 finite_field_matrix_rank,
                                 finite_field_inverse,
                                 is_prime)

from numpy.typing import NDArray

from coherentinfo.errormodel import ErrorModel, ErrorModelPoisson
# import galois
import scipy
import timeit

In [2]:
length = 5
width = 5
p = 3
moebius_code = MoebiusCode(length=length, width=width, d=2 * p)
h_z = moebius_code.h_z
h_x = moebius_code.h_x
logical_x = moebius_code.logical_x
logical_z = moebius_code.logical_z

# Vertex syndrome

Now we want to understand how to construct a candidate vector given a certain vertex syndrome. We start with a basic example. Note that the syndrome is mod 2 * p

In [3]:
for _ in range(1000):
    error = np.random.randint(2 * p, size=moebius_code.num_edges)
    syndrome = moebius_code.h_z @ error.T % (2 * p)
    candidate_error = moebius_code.get_vertex_candidate_error(syndrome)
    syndrome_candidate = moebius_code.h_z @ candidate_error.T % (2 * p)
    if np.count_nonzero(syndrome - syndrome_candidate) != 0:
        print("Syndromes do not match")
        break

In [4]:
error = np.random.randint(2 * p, size=moebius_code.num_edges)
candidate_error = moebius_code.get_vertex_candidate_error(syndrome)
error_diff = error - candidate_error
res_com = error_diff @ logical_z.T % (2 * p)
print(res_com == 0 or res_com == p)

True


# Plaquette Syndrome

Now we want to understand how to construct a candidate vector given a certain plaquette syndrome. We start with a basic example. Note that the syndrome is mod 2 * p

In [5]:
for _ in range(100):
    error = np.random.randint(2 * p, size=moebius_code.num_edges)
    syndrome = moebius_code.h_x @ error.T % (2 * p)
    candidate_error = moebius_code.get_plaquette_candidate_error(syndrome)
    syndrome_candidate = moebius_code.h_x @ candidate_error.T % (2 * p)
    if np.count_nonzero(syndrome - syndrome_candidate) != 0:
        print("Syndromes do not match")
        break

error = np.random.randint(2 * p, size=moebius_code.num_edges)
candidate_error = moebius_code.get_plaquette_candidate_error(syndrome)
error_diff = error - candidate_error
print(error_diff @ logical_x.T % (2 * p))

3


# Error model

In [6]:
gamma = 0.1
num_errors = 1000000
poisson_em = ErrorModelPoisson(num_errors, 2 * p, gamma)

In [7]:
probs = poisson_em.get_probabilities()
print(probs)

[8.18730753e-01 1.63746151e-01 1.63746151e-02 1.09164100e-03
 5.45820502e-05 2.18328201e-06]


In [24]:
my_error = poisson_em.generate_random_error()

In [25]:
sampled_frequencies = np.array([
    np.argwhere(my_error == x).shape[0] / num_errors
    for x in range(2 * p)
])
print(sampled_frequencies)

[8.18732e-01 1.63972e-01 1.61140e-02 1.11400e-03 6.70000e-05 1.00000e-06]
